# Computational and Systems Biology
## $\texttt{Analysis of food pairing and food bridging hypotheses}$

Authors:    
1. Vishwa Shah (201801036)
2. Riddhi Tanna (201801427)
3. Dishita Thaker (201801442)

This notebook contains the code for preprocessing data and obtaining the metric and semi-metric backbone networks. 

###▶ Semi metric edge:
An edge between two nodes is said to be semi-metric if there does exists an alternative path that is shorter than the direct edge between them. 

###▶  Metric edge:
An edge between two node is said to be metric if there does not exist an alternative path that is shorter than the direct edge between them. 

--- 
In our case, a path is shorter if there is a strong connection through shared flavours. Hence, we need to find the longest path between the two nodes. But, this problem is NP-complete and hence, is very expensive computationally. So, we resort to comparing the direct edges with the first 10 shortest paths between two nodes. If the cost of any of those paths is more than the cost of the direct edge, we classify that edge as a semi-metric edge. Otherwise, the edge is metric. 

--- 

### Importing libraries


In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import random

### Mounting drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Loading datasets

In [ ]:
# change path here
data_fb = pd.read_csv('/content/drive/Shareddrives/FoodBridging/flavor_network_backbone/favor_backbone.csv')
recipe_ing = pd.read_csv('/content/drive/Shareddrives/FoodBridging/flavor_network_backbone/recipe_ingredient.csv', header=None)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (30,31,32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
recipe_ing[0].unique()

array(['African', 'EastAsian', 'EasternEuropean', 'LatinAmerican',
       'MiddleEastern', 'NorthAmerican', 'NorthernEuropean', 'SouthAsian',
       'SoutheastAsian', 'SouthernEuropean', 'WesternEuropean'],
      dtype=object)

In [ ]:
cuisines = ['EastAsian', 'LatinAmerican', 'EasternEuropean', 'WesternEuropean', 'SouthernEuropean', 'SoutheastAsian', 'NorthAmerican']

### Data preprocessing

In [ ]:
df_reduced = pd.DataFrame()
for cuisine in cuisines: 
    idxs = random.sample(list(recipe_ing.index[recipe_ing[0] == cuisine]), 20)
    df_reduced = df_reduced.append(recipe_ing.iloc[idxs], ignore_index=True)

### Reduced dataframe

In [ ]:
df_reduced

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32
0,EastAsian,nut,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EastAsian,sesame_oil,roasted_sesame_seed,soy_sauce,cayenne,chinese_cabbage,shrimp,garlic,ginger,scallion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EastAsian,sesame_oil,mushroom,starch,shallot,corn,ginger,white_wine,bean,carrot,garlic,soybean,oyster,cilantro,onion,asparagus,chicken_broth,celery,fish,soy_sauce,root,shiitake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,EastAsian,garlic,fish,cayenne,soy_sauce,potato,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EastAsian,garlic,sesame_oil,roasted_sesame_seed,black_pepper,soy_sauce,cayenne,scallion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,NorthAmerican,tomato,olive_oil,onion,vinegar,black_pepper,parmesan_cheese,macaroni,bell_pepper,basil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
136,NorthAmerican,tomato,dill,chicken,onion,turmeric,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
137,NorthAmerican,butter,wheat,bell_pepper,onion,potato,parmesan_cheese,bread,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138,NorthAmerican,butter,pepper,macaroni,cream_cheese,milk,parmesan_cheese,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_r2 = df_reduced.drop([0], axis = 1)

### Saving the reduced df

In [ ]:
# change path here
df_reduced.to_csv('/content/drive/Shareddrives/FoodBridging/data/reduced_df_20.csv')

### Finding unique ingredients from the reduced sample to make a subgraph of the flavour network

In [ ]:
unique_nodes = pd.Series(df_r2.values.ravel()).unique()
print('Number of unique nodes in the sub graph of the sample: ', len(pd.Series(df_r2.values.ravel()).unique()))

Number of unique nodes in the sub graph of the sample:  188


In [ ]:
unique_nodes

array(['nut', nan, 'sesame_oil', 'roasted_sesame_seed', 'soy_sauce',
       'cayenne', 'chinese_cabbage', 'shrimp', 'garlic', 'ginger',
       'scallion', 'mushroom', 'starch', 'shallot', 'corn', 'white_wine',
       'bean', 'carrot', 'soybean', 'oyster', 'cilantro', 'onion',
       'asparagus', 'chicken_broth', 'celery', 'fish', 'root', 'shiitake',
       'potato', 'black_pepper', 'cane_molasses', 'peanut', 'almond',
       'raisin', 'walnut', 'cinnamon', 'vegetable_oil', 'sesame_seed',
       'vinegar', 'vegetable', 'egg', 'chicken', 'beef_broth',
       'watermelon', 'sake', 'honey', 'matsutake', 'pork',
       'green_bell_pepper', 'celery_oil', 'tomato', 'cider', 'pineapple',
       'beef', 'pear', 'wheat', 'rice', 'radish', 'parsley',
       'japanese_plum', 'cucumber', 'seaweed', 'tuna', 'butter', 'wine',
       'clam', 'peanut_oil', 'yeast', 'oregano', 'tequila', 'lime_juice',
       'cumin', 'cheese', 'cottage_cheese', 'bell_pepper', 'olive_oil',
       'cassava', 'avocado', 'c

In [ ]:
data_fb.columns = ['source', 'target', 'weight']

In [ ]:
data_fb[['weight']].apply(pd.to_numeric)

,weight
0,3
1,5
2,57
3,1
4,2
...,...
221772,1
221773,6
221774,1
221775,6


### Constructing the backbone networks

In [ ]:
G = nx.Graph()
G = nx.from_pandas_edgelist(data_fb, edge_attr='weight')

In [ ]:
# making a sampled subgraph 
G_sub = G.subgraph(unique_nodes)

In [ ]:
[(u, v, d) for (u, v, d) in G_sub.edges(data=True) if u == 'ginger' and v == 'garlic']

[('ginger', 'garlic', {'weight': 2})]

In [ ]:
# plt.figure(figsize = [100,100])
# pos = nx.circular_layout(G_sub)

# nx.draw(G_sub, pos, with_labels= True)

In [ ]:
from itertools import islice
def k_shortest_paths(G, source, target, k, weight = 'weight'):
  '''
  Function for k-shortest paths 
  '''
        return list(islice(nx.shortest_simple_paths(G, source, target, weight = 'weight'), k))

In [ ]:
edges = list(G_sub.edges)

In [ ]:
G_sub.get_edge_data(edges[0][0], edges[0][1])['weight']

6

In [ ]:
from tqdm.notebook import trange, tqdm
m = []
sm = []

for i in trange(len(edges)):
    flag = 1
    edge = edges[i]
    source_vertex = edge[0]
    target_vertex = edge[1]

    ksp = k_shortest_paths(G_sub, source = source_vertex, target = target_vertex, k = 10)
    weight = G_sub.get_edge_data(edges[i][0], edges[i][1])['weight']

    sp_costs = []

    for i in range(len(ksp)):
      path_cost = 0
      for j in range(len(ksp[i]) - 1):
        path_cost = path_cost + G_sub.get_edge_data(ksp[i][j], ksp[i][j+1])['weight']

      #print(path_cost, weight)
      sp_costs.append(path_cost)

    
    for cost in sp_costs: 
        if flag == 1:
            if cost > weight:
                flag = 0
                sm.append(edge)
                #print('semi-metric', len(sm))
                break

    if flag == 1:
        m.append(edge)
        #print('metric', len(m))

  0%|          | 0/11173 [00:00<?, ?it/s]

In [ ]:
len(sm), len(m)

(2708, 8465)

### Saving the backbone networks

In [ ]:
semi_metric = G_sub.edge_subgraph(sm).copy()

In [ ]:
metric = G_sub.edge_subgraph(m).copy()

In [ ]:
nx.write_edgelist(semi_metric, '/content/drive/Shareddrives/FoodBridging/data/semi_metric_20.csv', comments="#", delimiter=",", data=True, encoding="utf-8")

In [ ]:
nx.write_edgelist(metric, '/content/drive/Shareddrives/FoodBridging/data/metric_20.csv', comments="#", delimiter=",", data=True, encoding="utf-8")